In [ ]:
import pandas as pd
import os
import numpy as np
import random
from matplotlib import pyplot as plt
from scipy.optimize import minimize
import copy

from experiment_params import *
from cost_funcs import *
from fl_sim_classes import *
import time
import pickle
from sklearn.decomposition import PCA

In [ ]:
path = r'C:\Users\kdmen\Desktop\Research\personalization-privacy-risk\Data'
cond0_filename = r'\cond0_dict_list.p'
all_decs_init_filename = r'\all_decs_init.p'
nofl_decs_filename = r'\nofl_decs.p'
id2color = {0:'lightcoral', 1:'maroon', 2:'chocolate', 3:'darkorange', 4:'gold', 5:'olive', 6:'olivedrab', 
            7:'lawngreen', 8:'aquamarine', 9:'deepskyblue', 10:'steelblue', 11:'violet', 12:'darkorchid', 13:'deeppink'}
implemented_client_training_methods = ['EtaGradStep', 'EtaScipyMinStep', 'FullScipyMinStep']
implement_these_methods_next = ['APFL', 'AFL', 'PersA_FL_MAML', 'PersA_FL_ME', 'PFA']
num_participants = 14

# For exclusion when plotting later on
bad_nodes = [1,3,13]

with open(path+cond0_filename, 'rb') as fp:
    cond0_training_and_labels_lst = pickle.load(fp)

D_0_7 = np.random.rand(2,7)

# PARS-Push: Personalized, Robust Stochastic Gradient-Push
- We consider a static, directed, and strongly connected network G = {[n], E} with no self-loops, and E ⊆ [n] × [n], where (i, j) ∈ E if there is an edge from node i to j. --> Eg this is not federated learning, appears to be fully distributed
- The core of our method relies on the running sum technique developed in [20]–[22] which is equivalent to the Push-Sum algorithm [25] over a sequence of virtual time-varying augmented graphs. 
- PARS-Push has two key aspects: (i) robust asynchronous aggregation over a virtual, augmented graph, and (ii) stochastic gradient descent with respect to unbiased stochastic gradients of (4).
- In-neighbors are the adjacent nodes that are directed IN to the target agent i, and out-neighbors are just the reverse (the nodes that the target agent points to).
- How to adapt distributed layout to FL case?

1. First, agent i select u+1 independent data batches curly_v wrt p_i, and after performing u steps of stochastic gradient descent starting from zi (Lines 6-11)

1. Agent i computes an unbiased stochastic gradient of (4), and then updates xi in Line 12

1. Second, node i updatesits parameters xi, yi, φx i , and φy i according to Line 14-15, 

1. Then sends the running sum parameters to its out-neighbors N_i^+ (Line 16)

1. Finally, agent i processes the received messages from its in-neighbors N − i which leads to selecting the most recent updates (Lines 17-22)

1. Consequently, agent i updates xi, yi, and zi in Lines 23-25 by combining newly received messages.


In [ ]:
elif self.global_method=='APFL': 
    t = self.current_global_round  # Should this be global or local? Global based on how they wrote it...
    # F.T@F is not symmetric, so use eig not eigh
    # eig returns (UNORDERED) eigvals, eigvecs
    if self.use_real_hess:
        # May be better to check to see if the update is in the update_transition_log...
        # Should also just be able to use self.current_update%self.local_round_threshold or something
        if self.prev_update == self.current_update:  # When is this condition ever true......
            # Note that this changes if you want to do SGD instead of GD
            eigvals = self.prev_eigvals
        else:
            print(f"Client{self.ID}: Recalculating the Hessian for new update {self.current_update}!")
            eigvals, _ = np.linalg.eig(hessian_cost_l2(self.F, self.alphaD))
            self.prev_eigvals = eigvals
            # Coping mechanism
            self.prev_update = self.current_update
    mu = np.amin(eigvals)  # Mu is the minimum eigvalue
    if mu.imag < self.tol and mu.real < self.tol:
        raise ValueError("mu is ~0, thus implying func is not mu-SC")
    elif mu.imag < self.tol:
        mu = mu.real
    elif mu.real < self.tol:
        print("Setting to imaginary only")  # This is an issue if this runs
        mu = mu.imag
    L = np.amax(eigvals)  # L is the maximum eigvalue
    if L.imag < self.tol and L.real < self.tol:
        raise ValueError("L is 0, thus implying func is not L-smooth")
    elif mu.imag < self.tol:
        L = L.real
    elif L.real < self.tol:
        print("Setting to imaginary only")  # This is an issue if this runs
        L = L.imag
    if self.verbose: 
        # Find a better way to print this out without spamming the console... eg log file...
        print(f"Client{self.ID}: L: {L}, mu: {mu}")
    kappa = L/mu
    a = np.max([128*kappa, self.tau])
    eta_t = 16 / (mu*(t+a))
    if self.input_eta:
        if self.safe_lr_factor!=False:
            raise ValueError("Cannot input eta AND use safe learning rate (they overwrite each other)")
        eta_t = self.eta
    elif self.safe_lr_factor!=False:
        print("Forcing eta_t to be based on the input safe lr factor")
        # This is only subtly different from just inputting eta... a little more dynamic ig
        eta_t = 1/(self.safe_lr_factor*L)
    elif eta_t >= 1/(2*L):
        # Note that we only check when automatically setting
        # ie if you manually input it will do whatever you tell it to do
        raise ValueError("Learning rate is too large according to constaints on GD")
    if self.verbose:
        print(f"Client{self.ID}: eta_t: {eta_t}")
    self.p.append((t+a)**2)
    if self.track_lr_comps:
        self.L_log.append(L)
        self.mu_log.append(mu)
        self.eta_t_log.append(eta_t)

    if self.adaptive:
        self.adap_alpha.append(self.adap_alpha[-1] - eta_t*np.inner(np.reshape((self.w-self.global_w), (self.PCA_comps*2)), np.reshape(gradient_cost_l2(self.F, self.mixed_w, self.H, self.Vmixed, self.learning_batch, self.alphaF, self.alphaD, Ne=self.PCA_comps), (2*self.PCA_comps))))
        # This is theoretically the same but I'm not sure what grad_alpha means
        #self.sus_adap_alpha.append() ... didn't write yet

    # GRADIENT DESCENT BASED MODEL UPDATE
    # NOTE: eta_t IS DIFFERENT FROM CLIENT'S ETA (WHICH IS NOT USED)
    # Why do I flatten the grads...

    global_gradient = np.reshape(gradient_cost_l2(self.F, self.global_w, self.H, self.Vglobal, self.learning_batch, self.alphaF, self.alphaD, Ne=self.PCA_comps), (2, self.PCA_comps))
    local_gradient = np.reshape(gradient_cost_l2(self.F, self.mixed_w, self.H, self.Vmixed, self.learning_batch, self.alphaF, self.alphaD, Ne=self.PCA_comps), (2, self.PCA_comps))
    # Gradient clipping
    if self.gradient_clipping:
        if np.linalg.norm(global_gradient) > self.clipping_threshold:
            global_gradient = self.clipping_threshold*global_gradient/np.linalg.norm(global_gradient)
        if np.linalg.norm(local_gradient) > self.clipping_threshold:
            local_gradient = self.clipping_threshold*local_gradient/np.linalg.norm(local_gradient)
    if self.track_gradient:
        self.gradient_log.append(np.linalg.norm(gradient_cost_l2(self.F, self.w, self.H, self.V, self.learning_batch, self.alphaF, self.alphaD, Ne=self.PCA_comps)))
        self.pers_gradient_log.append(np.linalg.norm(local_gradient))
        # ^ Local gradient is evaluated wrt mixed inputs (eg w and V)
        self.global_gradient_log.append(np.linalg.norm(global_gradient))

    ########################################
    # Or should I normalize the dec here?  I'll also turn this on since idc about computational speed rn
    if self.normalize_dec:
        self.global_w /= np.amax(self.global_w)
        self.w /= np.amax(self.w)
        self.mixed_w /= np.amax(self.mixed_w)
    ########################################

    # PSEUDOCODE: my_client.global_w -= my_client.eta * grad(f_i(my_client.global_w; my_client.smallChi))
    self.global_w -= eta_t * global_gradient
    # PSEUDOCODE: my_client.local_w -= my_client.eta * grad_v(f_i(my_client.v_bar; my_client.smallChi))
    self.w -= eta_t * local_gradient
    self.mixed_w = self.adap_alpha[-1]*self.w - (1 - self.adap_alpha[-1])*self.global_w
    ########################################
    # Or should I normalize the dec here?  I'll also turn this on since idc about computational speed rn
    if self.normalize_dec:
        self.global_w /= np.amax(self.global_w)
        self.w /= np.amax(self.w)
        self.mixed_w /= np.amax(self.mixed_w)

In [ ]:
 ## Inputs
#x_i  # exists in R^d; i is each node
#theta_t  # exists in R for all t existing in Z0^+
# Init
y_i = 1
z_i = x_i
kappa_i = -1
phi_i_x = 0
phi_i_y = 0
kappa_ij = -1  # For all (j, i) existing in Edges
rho_ij_x = -1  # For all (j, i) existing in Edges
rho_ij_y = -1  # For all (j, i) existing in Edges
for t in range(1000):
    for my_client in all_clients:
        if my_client.available:
            # Line 5
            my_client.n = np.sum(theta)  # From my_client.kappa+1 to t
            # Line 6-11
            w_i = z_i
            for r in range(u-1):
                # Sample a batch D^t_{i, r} with size b from p_i
                w_i^(r+1) = w_i^r - alpha*nabla_tilde(w_i^r, D^t_{i, r})
            # Sample a batch D^t_{i, u} with size b from p_i
            # Is that an introduction or just now shown ...
            # Line 12
            # Crazy eqn
            x_i = x_i - n_i[...]
            kappa_i = t
            # Line 14-15
            x_i = x_i/(d^+_i + 1)
            y_i = y_i/(d^+_i + 1)
            phi^x_i += x_i
            phu^y_i += y_i
            # Line 16
            blah
            # Lines 17-22
            blah
            for (phi^x_i, phi^y_i, kappa_i) in R_i:
                if kappa_j > kappa_{ij}:
                    blahx
                    blahy
            # Line 23
            blahx
            blahy
            blahx
            blahy
#return z_i 
